# Modeling Men's National Football (soccer) Team scores in World Cup 2022

In this notebook we develop a model for soccer performance and implement a Bayesian model via STAN.

I hope this is an interesting introduction to 
- Bayes Statistics
- Data science programming
- STAN

Originally I wanted to make a full length tutorial on this topic, but due to my limited bandwidth I'm only writing this notebook. Here we go!

In [ ]:
# install packages on Colab
!pip3 install pystan
!pip3 install nest_asyncio

In [ ]:
# because jupyter doesn't allow asyncio for pystan, we need this config
import nest_asyncio
nest_asyncio.apply()

In [ ]:
import pandas as pd
import numpy as np
import stan 
import pickle

## Import DATA
source: [Kaggle soccer result data set](https://www.kaggle.com/datasets/martj42/international-football-results-from-1872-to-2017)
We download the dataset and upload them to the source folder of this colab notebook. Alternatively we can setup a API link with our kaggle account, but since the csv files are so small I'd just upload them manually. 

*NOTE: Colab does not pertain the files folder. Once we disconnect from the host machine, everything except for the notebook will be wiped out. The best practice is to keep data ELSEWHERE.*

In this notebook, I linked colab with my google drive because later on I need to pickle (means save) my model in a safe space. 

In [ ]:
# Read data
goalscorers = pd.read_csv("goalscorers.csv")
results = pd.read_csv("results.csv")
shootouts = pd.read_csv("shootouts.csv")
# ranking = pd.read_csv("fifa_ranking-2022-10-06.csv")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/My\ Drive/WC2022

/content/drive/My Drive/WC2022


In [ ]:
print("total number of countries:",len(set(results.home_team.values)))
print("total number of games:",results.shape[0])

total number of countries: 311
total number of games: 44152


In [ ]:
# look at the data
results.head()

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
0,1872-11-30,Scotland,England,0.0,0.0,Friendly,Glasgow,Scotland,False
1,1873-03-08,England,Scotland,4.0,2.0,Friendly,London,England,False
2,1874-03-07,Scotland,England,2.0,1.0,Friendly,Glasgow,Scotland,False
3,1875-03-06,England,Scotland,2.0,2.0,Friendly,London,England,False
4,1876-03-04,Scotland,England,3.0,0.0,Friendly,Glasgow,Scotland,False


## Statistical Modeling

<!-- zero-inflated Poisson
$$\text{Score}_1 = \begin{cases}0&\text{w.p. } \pi_1\\\text{Pois}\left(a_1-d_2 +m_1R_{1}+ h_1\mathbb{1}_{(\text{1 at home})} - h_2\mathbb{1}_{(\text{2 at home})}\right)& \text{w.p. } 1-\pi_1 \end{cases}$$ -->

### Poisson Distribution

Football (soccer) is a game between 2 teams, each aiming to score as many goals as possible in 90 minutes. 
For each score, we can consider a Poisson model. A poisson model is a model for integers $0,1,\ldots$ parameterized by "rate". In plain words, it represents "if we know that some event (say, scoring)  would occur $\lambda$ times in a given interval (say, 90 minute game), what is the distribution of number of occurrance you would observe". Well, there are other assumptioned built into the model, but this is the general idea, and I would say this matches perfertly with out intuition about a football game! Say, if the rate is 3 goals per game, under Poisson distribution, there is a 5% chance to score 0 goal, 15% chance for 1 goal, 22% chance for 2 goals, etc. 

### Poisson Regression formulation

I mentioned Poisson has one parameter, the rate. Now let's think about what is the "rate" in football's sense: It is the expected number of goals a team scores in a match. In fact, expected number of goals (called, xG in most soccer stats places) makes a pretty good model for prediction. 
However, many things can affact the ability of teams scoring goals, say, who is the team playing against (usually more xG against weaker opponants )? is the team playing home or away? 

In light of this, we express the rate as a linear combination of a few factors. 
If Team 1 is at home, Team 2 is away, then:
$$\text{Score}_1 \sim \text{Pois}\left(a_1-d_2 +m_1R_{1}+ h_{1}^{(a)} \right)$$
$$
\text{Score}_2 \sim \text{Pois}\left(a_2-d_1 +m_2R_{2}- h_{1}^{(d)} \right)
$$

Parameters
- $a$: attack strength
- $d$: defend strength
- $h$: home performance bonus (attack and defend)
- $m$: momentum, how much does past performance influence this game
- $R$: average points of past 10 games

# Data Cleansing
Next we cleanse the data to feed into out model. 
Our goal is to only keep relavent information and add additional info. 
One feature we would like to include is the "recent form" of each team before each game. The belief is that form matters: a team with winning streak usually has a mental advantage walking into the stadium. 

In [ ]:
# all 32 teams in WC2022
teams = ["Qatar", "Ecuador", "Senegal", "Netherlands",
         "England", "Iran", "United States", "Wales", 
         "Argentina", "Saudi Arabia", "Mexico", "Poland", 
         "France", "Australia", "Denmark", "Tunisia",
         "Spain", "Costa Rica", "Germany", "Japan",
         "Belgium", "Canada", "Morocco", "Croatia",
         "Brazil", "Serbia", "Switzerland", "Cameroon",
         "Portugal", "Ghana", "Uruguay", "South Korea"]

In [ ]:
# Make recent form
recent_point_average = dict.fromkeys(teams,[]) # use a dict as FIFO stack, keep 10 items
recent_form = []
for index, row in results[~results.home_score.isna()].iterrows():
  form = [None, None] # form of home and away team
  # get score
  home_pt = 3*(row.home_score>row.away_score)+1*(row.home_score==row.away_score)
  away_pt = 3*(row.home_score<row.away_score)+1*(row.home_score==row.away_score)
  # we only record form of the 32 WC teams
  if row.home_team in teams:
    form[0]=np.mean(recent_point_average[row.home_team])
    recent_point_average[row.home_team]=recent_point_average[row.home_team]+[home_pt]
    # print(row.home_team,recent_point_average[row.home_team],np.mean(recent_point_average[row.home_team]))
    if len(recent_point_average[row.home_team])>10:
      recent_point_average[row.home_team].pop(0)
  if row.away_team in teams:
    form[1]=np.mean(recent_point_average[row.away_team])
    recent_point_average[row.away_team]=recent_point_average[row.away_team]+[away_pt]
    # print(row.away_team,recent_point_average[row.away_team],np.mean(recent_point_average[row.away_team]))
    if len(recent_point_average[row.away_team])>10:
      recent_point_average[row.away_team].pop(0)
  recent_form.append(form)

/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3441: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [ ]:
# make new feature
results["home_form"]=0
results["away_form"]=0
results.loc[~results.home_score.isna(),"home_form"] = np.asarray(recent_form)[:,0]
results.loc[~results.home_score.isna(),"away_form"] = np.asarray(recent_form)[:,1]

In [ ]:
# filter and only keep relevant games
results_within_32 = results[(results.home_team.isin(teams)) & 
                            (results.away_team.isin(teams)) & 
                            (~results.home_score.isna())].copy()

In [ ]:
# remove really old games
results_within_32=results_within_32[results_within_32.date>'1960-01-01']

In [ ]:
# make a df of games we want to predict, starting from group stage of WC2022
results_new=results[(results.country=="Qatar") & (results.tournament=="FIFA World Cup")].copy()

In [ ]:
results_new.loc[:,"home_form"]=[np.mean(recent_point_average[x]) for x in results_new.home_team]
results_new.loc[:,"away_form"]=[np.mean(recent_point_average[x]) for x in results_new.away_team]

In [ ]:
# look at training set now
results_within_32.head()

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,home_form,away_form
4665,1960-02-28,Belgium,France,1.0,0.0,Friendly,Brussels,Belgium,False,0.6,1.9
4667,1960-03-06,Brazil,Mexico,2.0,2.0,Pan American Championship,San José,Costa Rica,True,2.2,2.0
4668,1960-03-08,Costa Rica,Argentina,0.0,0.0,Pan American Championship,San José,Costa Rica,False,1.7,2.0
4669,1960-03-10,Argentina,Mexico,3.0,2.0,Pan American Championship,San José,Costa Rica,True,1.8,1.8
4670,1960-03-10,Costa Rica,Brazil,3.0,0.0,Pan American Championship,San José,Costa Rica,False,1.5,2.0


We can print the recent forms of the 32 teams coming into WC2022. Looks like Argentina and Brazil has the best form, averaging 2.6 points in the past 10 games. Wales is in poor form right now, aceraging only 0.9 pts. 

In [ ]:
# Print recent forms of the 32 teams
[{key:np.mean(val)} for key,val in recent_point_average.items()]

[{'Qatar': 1.8},
 {'Ecuador': 1.3},
 {'Senegal': 2.3},
 {'Netherlands': 2.4},
 {'England': 1.5},
 {'Iran': 1.9},
 {'United States': 1.6},
 {'Wales': 0.9},
 {'Argentina': 2.6},
 {'Saudi Arabia': 1.1},
 {'Mexico': 1.4},
 {'Poland': 1.4},
 {'France': 1.7},
 {'Australia': 1.6},
 {'Denmark': 1.8},
 {'Tunisia': 2.0},
 {'Spain': 2.3},
 {'Costa Rica': 2.5},
 {'Germany': 1.7},
 {'Japan': 2.0},
 {'Belgium': 1.8},
 {'Canada': 1.6},
 {'Morocco': 1.8},
 {'Croatia': 2.3},
 {'Brazil': 2.6},
 {'Serbia': 2.2},
 {'Switzerland': 1.4},
 {'Cameroon': 1.5},
 {'Portugal': 1.7},
 {'Ghana': 1.1},
 {'Uruguay': 2.2},
 {'South Korea': 2.0}]

## Fitting a Bayesian Model
In general, there are two ways to go about statistical modeling and sampling. We can either estimate the parameters and use that to generate new samples, or go the Bayesian way, and sample from the posterior which is a generative model. In simplified terms, the first route (called frequentist approach) assumes there are some true numbers (say, true xG of a game) that we can estimate through data; the second way (called Bayesian) we start from some knowledge (called prior), compute the probability of data given this knowledge (called likelihood) and use the Bayes formula to get the distribution of parameters given data (called posterior). 

Which way is better? It depends on how you think about the problem, and to be honest, statistics in general. Let's skip that talk and jump into the details. In this note we apply a Bayesian framework. We assume each parameter to have a non-informative prior, then sample the posterior, and use the posterior distribution to generate, guess what, score predictions!

## STAN, PySTAN
STAN is just a fast (really?), reliable (maybe?), readable (yes) and simple (definitely not) way to code Bayesian models. The thing for Bayesian models is this: these models are sort of "easy" to design, you just specify a bunch of relations between parameter and variables of interest, but they are really hard to compute because you need to calculate a gigantic normalizing constant, which is usually complicated and without closed form. The 
sampling method everyone is using is called MCMC, which allows fast sampling. 
Packages like STAN and HMC helps performing MCMC faster with various tricks. 

STAN is written in `C++` and primarily supported for the `R` community, the package is called RSTAN. People need to use STAN in other environment, so there's also PySTAN. In PySTAN, we need to specify a STAN model and data structure, then let the package do its magic. 

In [ ]:
# structured data
results_data = {"N": results_within_32.shape[0],
                "M": results_new.shape[0],
                "p":32,
                "T1":np.asarray([teams.index(x)+1 for x in results_within_32.home_team]), 
                "T2":np.asarray([teams.index(x)+1 for x in results_within_32.away_team]), 
                "home": np.ones(results_within_32.shape[0])*results_within_32.neutral.values,
                "recent_form":np.transpose(np.vstack([results_within_32.home_form.values,results_within_32.away_form.values])).astype(float),
                "score1": results_within_32.home_score.values.astype(int),
                "score2": results_within_32.away_score.values.astype(int),
                "T1new":np.asarray([teams.index(x)+1 for x in results_new.home_team]),
                "T2new":np.asarray([teams.index(x)+1 for x in results_new.away_team]), 
                "recent_form_new":np.transpose(np.vstack([results_new.home_form.values,results_new.away_form.values])).astype(float),
                }

In [ ]:
# STAN model
stan_code = """
data {
  // Define variables in data
  int<lower=0> N;
  int<lower=0> M;
  int<lower=0> p;
  int<lower=1, upper=p> T1[N];
  int<lower=1, upper=p> T2[N];
  int<lower=1, upper=p> T1new[M];
  int<lower=1, upper=p> T2new[M];

  real home[N];
  vector[2] recent_form[N];
  vector[2] recent_form_new[M];
  
  // outcome
  int<lower=0> score1[N];
  int<lower=0> score2[N];
}

parameters {
  // Define parameters to estimate
  real a[p];
  real d[p];
  real <lower=0, upper=1> m[p];
  real[2] <lower=0, upper=1> h[p];
}

transformed parameters  {
  // parameters
  real lp1[N];
  real lp2[N];
  real <lower=0> mu1[N];
  real <lower=0> mu2[N];

  for (i in 1:N) {
    // Linear predictor
    lp1[i] = a[T1[i]] - d[T2[i]] + h[T1[i],1]*home[i] + m[T1[i]]*recent_form[i,1];
    lp2[i] = a[T2[i]] - d[T1[i]] - h[T1[i],2]*home[i] + m[T2[i]]*recent_form[i,2];

    // Mean
    mu1[i] = exp(lp1[i]);
    mu2[i] = exp(lp2[i]);
  }
}

model {
  // Prior part of Bayesian inference
  // Likelihood part of Bayesian inference
  score1 ~ poisson(mu1);
  score2 ~ poisson(mu2);
}
generated quantities {
    int<lower=0> group_stage_score[M,2];
    real mu1new[M];
    real mu2new[M];
    real lp1new[M];
    real lp2new[M];

    for (i in 1:M) {
      lp1new[i] = a[T1new[i]] - d[T2new[i]] + m[T1new[i]]*recent_form_new[i,1];
      lp2new[i] = a[T2new[i]] - d[T1new[i]] + m[T2new[i]]*recent_form_new[i,2];
      mu1new[i] = exp(lp1new[i]);
      mu2new[i] = exp(lp2new[i]);
      group_stage_score[i,1] = poisson_rng(mu1new[i]);
      group_stage_score[i,2] = poisson_rng(mu2new[i]);
    }
}
"""

In [ ]:
# build model, this step checks if the model is valid
posterior = stan.build(stan_code, data=results_data)

Building...



Building: found in cache, done.Messages from stanc:


In [ ]:
# fit the model with data
fit = posterior.sample(num_chains=4, num_samples=1000)

Sampling:   0%
Sampling:   0% (1/8000)
Sampling:   1% (100/8000)
Sampling:   2% (200/8000)
Sampling:   4% (300/8000)
Sampling:   5% (400/8000)
Sampling:   5% (401/8000)
Sampling:   6% (501/8000)
Sampling:   8% (600/8000)
Sampling:   9% (700/8000)
Sampling:  10% (800/8000)
Sampling:  11% (900/8000)
Sampling:  12% (1000/8000)
Sampling:  14% (1100/8000)
Sampling:  15% (1200/8000)
Sampling:  16% (1300/8000)
Sampling:  18% (1400/8000)
Sampling:  19% (1500/8000)
Sampling:  19% (1501/8000)
Sampling:  20% (1601/8000)
Sampling:  21% (1700/8000)
Sampling:  22% (1800/8000)
Sampling:  24% (1900/8000)
Sampling:  25% (2000/8000)
Sampling:  26% (2100/8000)
Sampling:  28% (2200/8000)
Sampling:  29% (2300/8000)
Sampling:  30% (2400/8000)
Sampling:  31% (2500/8000)
Sampling:  32% (2600/8000)
Sampling:  33% (2601/8000)
Sampling:  34% (2701/8000)
Sampling:  35% (2801/8000)
Sampling:  36% (2900/8000)
Sampling:  38% (3000/8000)
Sampling:  39% (3100/8000)
Sampling:  39% (3101/8000)
Sampling:  40% (3200/8000)

In [ ]:
# save model
with open("model_fit.pkl", "wb") as f:
    pickle.dump({'model' : posterior, 'fit' : fit}, f, protocol=-1)

In [ ]:
# load model
with open("model_fit.pkl", "rb") as f:
    data_dict = pickle.load(f)
fit = data_dict['fit']

## Play with the samples
Now we have 4000 generated samples of WC2022 group stage, that means we have hypothetically repeated the event 4000 times. Time to have some fun!

In [ ]:
# record pts, goals, goals against for all teams
sampled_pt = dict.fromkeys(teams,np.zeros(fit["group_stage_score"].shape[2]))
sampled_gf = dict.fromkeys(teams,np.zeros(fit["group_stage_score"].shape[2]))
sampled_ga = dict.fromkeys(teams,np.zeros(fit["group_stage_score"].shape[2]))

for ind, row in results_new.reset_index().iterrows():
  s_result = fit['group_stage_score'][ind,0,:]-fit['group_stage_score'][ind,1,:]
  sampled_pt[row.home_team] = sampled_pt[row.home_team] + 3*(s_result>0)+ 1*(s_result==0)
  sampled_pt[row.away_team] = sampled_pt[row.away_team] + 3*(s_result<0)+ 1*(s_result==0)
  sampled_gf[row.home_team] = sampled_gf[row.home_team] + fit['group_stage_score'][ind,0,:]
  sampled_ga[row.home_team] = sampled_ga[row.home_team] + fit['group_stage_score'][ind,1,:]
  sampled_gf[row.away_team] = sampled_gf[row.away_team] + fit['group_stage_score'][ind,1,:]
  sampled_ga[row.away_team] = sampled_ga[row.away_team] + fit['group_stage_score'][ind,0,:]

In [ ]:
groups = {
    "A": ["Qatar", "Ecuador", "Senegal", "Netherlands"],
    "B":["England", "Iran", "United States", "Wales", ],
    "C":["Argentina", "Saudi Arabia", "Mexico", "Poland", ],
    "D":["France", "Australia", "Denmark", "Tunisia",],
    "E":["Spain", "Costa Rica", "Germany", "Japan",],
    "F":["Belgium", "Canada", "Morocco", "Croatia",],
    "G":["Brazil", "Serbia", "Switzerland", "Cameroon",],
    "H":["Portugal", "Ghana", "Uruguay", "South Korea"],
}

In [ ]:
# compute probability of making out of group stage
group_pbs = dict.fromkeys(teams)
for gp in groups.keys():
  pbs = np.mean(np.argsort(np.vstack([sampled_pt[x] for x in groups[gp]])+
                     np.vstack([sampled_gf[x] for x in groups[gp]])*0.01 - 
                     np.vstack([sampled_ga[x] for x in groups[gp]])*0.01,axis=0).argsort(axis=0)>=2,axis=1)
  for i in range(4):
    group_pbs[groups[gp][i]]=pbs[i]

In [ ]:
np.argsort(np.vstack([sampled_pt[x] for x in groups[gp]])+
                     np.vstack([sampled_gf[x] for x in groups[gp]])*0.01 - 
                     np.vstack([sampled_ga[x] for x in groups[gp]])*0.01,axis=0)

array([[1, 1, 1, ..., 1, 3, 3],
       [2, 2, 2, ..., 3, 1, 1],
       [3, 0, 3, ..., 0, 2, 0],
       [0, 3, 0, ..., 2, 0, 2]])

In [ ]:
# probability of making out of group stage
group_pbs

{'Qatar': 0.1365,
 'Ecuador': 0.46075,
 'Senegal': 0.472,
 'Netherlands': 0.93075,
 'England': 0.8905,
 'Iran': 0.396,
 'United States': 0.39175,
 'Wales': 0.32175,
 'Argentina': 0.8215,
 'Saudi Arabia': 0.1245,
 'Mexico': 0.539,
 'Poland': 0.515,
 'France': 0.80075,
 'Australia': 0.26225,
 'Denmark': 0.6105,
 'Tunisia': 0.3265,
 'Spain': 0.84925,
 'Costa Rica': 0.148,
 'Germany': 0.80325,
 'Japan': 0.1995,
 'Belgium': 0.698,
 'Canada': 0.0865,
 'Morocco': 0.44375,
 'Croatia': 0.77175,
 'Brazil': 0.92225,
 'Serbia': 0.44075,
 'Switzerland': 0.41675,
 'Cameroon': 0.22025,
 'Portugal': 0.743,
 'Ghana': 0.201,
 'Uruguay': 0.63825,
 'South Korea': 0.41775}

## Other fun investigations

### individual match prediction
We can "predict" these by aggregating the samples. Note that we cannot aggregate by sample mean (because goals are integers), but we aggregate median or mode instead. 

In [ ]:
median_prediction = results_new[['home_team','away_team']].copy()
median_prediction.loc[:,'home_score_predict'] = np.median(fit['group_stage_score'],axis=2)[:,0]
median_prediction.loc[:,'away_score_predict'] = np.median(fit['group_stage_score'],axis=2)[:,1]

In [ ]:
median_prediction

,home_team,away_team,home_score_predict,away_score_predict
44104,Qatar,Ecuador,1.0,2.0
44105,Senegal,Netherlands,1.0,2.0
44106,England,Iran,1.0,0.0
44107,United States,Wales,1.0,1.0
44108,Argentina,Saudi Arabia,2.0,0.0
44109,Mexico,Poland,1.0,1.0
44110,Denmark,Tunisia,1.0,1.0
44111,France,Australia,2.0,0.0
44112,Germany,Japan,2.0,1.0
44113,Spain,Costa Rica,2.0,0.0


In [ ]:
# predict by mode = most frequent occurance
from scipy import stats as st
mode_prediction = results_new[['home_team','away_team']].copy()
mode_prediction.loc[:,'home_score_predict'] = st.mode(fit['group_stage_score'],axis=2)[0][:,0]
mode_prediction.loc[:,'away_score_predict'] = st.mode(fit['group_stage_score'],axis=2)[0][:,1]
mode_prediction

,home_team,away_team,home_score_predict,away_score_predict
44104,Qatar,Ecuador,0.0,1.0
44105,Senegal,Netherlands,0.0,1.0
44106,England,Iran,1.0,0.0
44107,United States,Wales,1.0,0.0
44108,Argentina,Saudi Arabia,2.0,0.0
44109,Mexico,Poland,1.0,1.0
44110,Denmark,Tunisia,1.0,1.0
44111,France,Australia,1.0,0.0
44112,Germany,Japan,2.0,0.0
44113,Spain,Costa Rica,2.0,0.0


In [ ]:
# predict by rounded mean
mean_prediction = results_new[['home_team','away_team']].copy()
mean_prediction.loc[:,'home_score_predict'] = np.round(np.mean(fit['group_stage_score'],axis=2)[:,0])
mean_prediction.loc[:,'away_score_predict'] = np.round(np.mean(fit['group_stage_score'],axis=2)[:,1])
mean_prediction

,home_team,away_team,home_score_predict,away_score_predict
44104,Qatar,Ecuador,1.0,2.0
44105,Senegal,Netherlands,1.0,2.0
44106,England,Iran,2.0,1.0
44107,United States,Wales,1.0,1.0
44108,Argentina,Saudi Arabia,2.0,1.0
44109,Mexico,Poland,1.0,1.0
44110,Denmark,Tunisia,2.0,1.0
44111,France,Australia,2.0,1.0
44112,Germany,Japan,2.0,1.0
44113,Spain,Costa Rica,2.0,1.0
